In [3]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import msgpack
from io import BytesIO
from tensorflow.python.lib.io import file_io
import logging
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, OneHotEncoder

from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib

# tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.13.1


In [4]:
# Read in saved files.
logging.info('Loading data.')
f = BytesIO(file_io.read_file_to_string('./data/wikimedia-personal-attacks/wikimedia-personal-attacks-embeddings.npy', binary_mode=True))
vocab = np.load(f)

# Load features and labels.
f = BytesIO(file_io.read_file_to_string('./data/wikimedia-personal-attacks/wikimedia-personal-attacks-data.bin', binary_mode=True))
data = msgpack.unpack(f, raw=False)

# Number of docs, which is also the number of observations or samples,
# and the number of rows of the input matrix.
if -1 < 0:
    num_docs = len(data)
else:
    # Subtract 1 to shift to zero-indexing.
    num_docs = 1

# Convert data to numpy arrays.
logging.info('Converting data to arrays.')

# For keeping number of words in longest document in data.
max_words = 0

# Create arrays to store docs and labels.
docs = []
labels = []

# Iterate over data to build arrays of docs and labels.
for i in range(num_docs):
    #sys.stdout.write("processing record %i of %i       \r" % (i + 1, num_docs))
    #sys.stdout.flush()

    # Get index of document.
    doc = data[i]['idx']

    # Retrieve document from saved data and cast to array.
    doc = [item for sublist in doc for item in sublist]

    # Add document to docs array.
    docs.append(doc)

    # Add label to label array at same index.
    labels.append(data[i]['label'])

    # Track maximum number of words in document.
    if len(doc) > max_words:
        max_words = len(doc)

del data
print()


# Label encoder.
#   Encode labels with value between 0 and n_classes-1,
#   so for example 1 to 5 star ratings become 0 to 4.
le = LabelEncoder()
y = le.fit_transform(labels)

# Binarize labels in one-vs-all fashion if three or more classes.
lb = LabelBinarizer()
y_bin = lb.fit_transform(y)

In [5]:
# Char used as padding elements.
pad_id = 0

# Pads all docs to the length of the longest doc using the pad_id char.
x_train = sequence.pad_sequences(docs, 
                                 maxlen=1000,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)

x_test = sequence.pad_sequences(docs, 
                                maxlen=1000,
                                truncating='post',
                                padding='post',
                                value=pad_id)

In [6]:
print(len(x_train))
print(len(x_test))

115841
115841


In [7]:
print('Unpadded length of first doc:\t', len(docs[0]))
print('Unpadded length of second doc:\t', len(docs[1]))
print('Padded len of first doc:\t', len(x_train[0]))
print('Padded len of second doc:\t', len(x_train[1]))
print('x_train shape:\t\t\t', x_train.shape)
print('x_test shape:\t\t\t', x_test.shape)
print()
print('115,841 documents each of length 1000.')

Unpadded length of first doc:	 421
Unpadded length of second doc:	 125
Padded len of first doc:	 1000
Padded len of second doc:	 1000
x_train shape:			 (115841, 1000)
x_test shape:			 (115841, 1000)

115,841 documents each of length 1000.


In [8]:
x_len_train = np.array([min(len(x), max_words) for x in docs])
x_len_test = np.array([min(len(x), max_words) for x in docs])
print('Length of original, unpadded docs:', x_len_train)

Length of original, unpadded docs: [421 125  64 ...  34 143  40]


In [9]:
# NOTE: the datasets provided by the train and eval functions have been limited to 10K docs
# for development.

def parser(x, length, y):
    features = {"x": x, "len": length}
    return features, y

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_train[0:10000], x_len_train[0:10000], y_bin[0:10000]))
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_test[10000:20000], x_len_test[10000:20000], y_bin[10000:20000]))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

In [15]:
model_dir = tempfile.mkdtemp()

column = tf.feature_column.categorical_column_with_identity('x', len(vocab))
classifier = tf.estimator.LinearClassifier(feature_columns=[column], model_dir=os.path.join(model_dir, 'bow_sparse'))

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/xf/68k__xt949d59nyyhmqwx4x40000gp/T/tmps7fo4sjd/bow_sparse', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a75be1748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/xf/68k__xt949d59nyyhmqwx4x40000gp/T/tmps7fo4sjd/bow_sparse', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a75be1748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
all_classifiers = {}
def train_and_evaluate(classifier):
    # Save a reference to the classifier to run predictions later
    all_classifiers[classifier.model_dir] = classifier
    classifier.train(input_fn=train_input_fn, steps=2500)
    eval_results = classifier.evaluate(input_fn=eval_input_fn)
    predictions = np.array([p['logistic'][0] for p in classifier.predict(input_fn=eval_input_fn)])
        
    # Reset the graph to be able to reuse name scopes
    tf.reset_default_graph() 
    # Add a PR summary in addition to the summaries that the classifier writes
    pr = summary_lib.pr_curve('precision_recall', predictions=predictions, labels=y_bin[10000:20000].astype(bool), num_thresholds=21)
    with tf.Session() as sess:
        writer = tf.summary.FileWriter(os.path.join(classifier.model_dir, 'eval'), sess.graph)
        writer.add_summary(sess.run(pr), global_step=0)
        writer.close()
#     # Un-comment code to download experiment data from Colaboratory
#     from google.colab import files
#     model_name = os.path.basename(os.path.normpath(classifier.model_dir))
#     ! zip -r {model_name + '.zip'} {classifier.model_dir}
#     files.download(model_name + '.zip')

In [119]:
train_and_evaluate(classifier)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Restoring parameters from /var/folders/xf/68k__xt949d59nyyhmqwx4x40000gp/T/tmpledcqvkb/bow_sparse/model.ckpt-5000


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [121]:
embedding_size = 50
word_embedding_column = tf.feature_column.embedding_column(column, dimension=embedding_size)
classifier = tf.estimator.DNNClassifier(
    hidden_units=[100],
    feature_columns=[word_embedding_column], 
    model_dir=os.path.join(model_dir, 'bow_embeddings'))
train_and_evaluate(classifier)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/xf/68k__xt949d59nyyhmqwx4x40000gp/T/tmpledcqvkb/bow_embeddings', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xba6d2e7f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/xf/68k__xt949d59nyyhmqwx4x40000gp/T/tmpledcqvkb/bow_embeddings', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xba6d2e7f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:Restoring parameters from /var/folders/xf/68k__xt949d59nyyhmqwx4x40000gp/T/tmpledcqvkb/bow_embeddings/model.ckpt-2500


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [124]:
head = tf.contrib.estimator.binary_classification_head()

def cnn_model_fn(features, labels, mode, params):    
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], len(vocab), embedding_size,
        initializer=params['embedding_initializer'])
    
    training = mode == tf.estimator.ModeKeys.TRAIN
    dropout_emb = tf.layers.dropout(inputs=input_layer, 
                                    rate=0.2, 
                                    training=training)

    conv = tf.layers.conv1d(
        inputs=dropout_emb,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    
    # Global Max Pooling
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    
    hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)
    
    dropout_hidden = tf.layers.dropout(inputs=hidden, 
                                       rate=0.2, 
                                       training=training)
    
    logits = tf.layers.dense(inputs=dropout_hidden, units=1)
    
    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        
    optimizer = tf.train.AdamOptimizer()
    
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())

    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits, 
        train_op_fn=_train_op_fn)
  
params = {'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)}
cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        model_dir=os.path.join(model_dir, 'cnn'),
                                        params=params)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/xf/68k__xt949d59nyyhmqwx4x40000gp/T/tmpledcqvkb/cnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xbb4c1e8d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/xf/68k__xt949d59nyyhmqwx4x40000gp/T/tmpledcqvkb/cnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xbb4c1e8d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [125]:
train_and_evaluate(cnn_classifier)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Restoring parameters from /var/folders/xf/68k__xt949d59nyyhmqwx4x40000gp/T/tmpledcqvkb/cnn/model.ckpt-2500


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [16]:
learning_rate = 0.00001
dropout = 0.5
num_filters = 300
embedding_size = 50
word_embedding_column = tf.feature_column.embedding_column(column, dimension=embedding_size)

head = tf.contrib.estimator.binary_classification_head()

def cnn_model_fn(features, labels, mode, params):    
    
    # Word embeddings.
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], len(vocab), embedding_size,
        initializer=params['embedding_initializer'])
    
    training = mode == tf.estimator.ModeKeys.TRAIN
    
    dropout_emb = tf.layers.dropout(inputs=input_layer, 
                                    rate=dropout, 
                                    training=training)
    
    # Convolutional layers.
    conv3 = tf.layers.conv1d(
        inputs=dropout_emb,
        filters=num_filters,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    pool3 = tf.reduce_max(input_tensor=conv3, axis=1)
    
    conv4 = tf.layers.conv1d(
        inputs=dropout_emb,
        filters=num_filters,
        kernel_size=4,
        padding="same",
        activation=tf.nn.relu)
    pool4 = tf.reduce_max(input_tensor=conv4, axis=1)
    
    conv5 = tf.layers.conv1d(
        inputs=dropout_emb,
        filters=num_filters,
        kernel_size=5,
        padding="same",
        activation=tf.nn.relu)
    pool5 = tf.reduce_max(input_tensor=conv5, axis=1)
    
    concat_pools = tf.concat([pool3, pool4, pool5], 1)
    
    hidden = tf.layers.dense(inputs=concat_pools, units=250, activation=tf.nn.relu)
    
    dropout_hidden = tf.layers.dropout(inputs=hidden, 
                                       rate=dropout, 
                                       training=training)
    
    logits = tf.layers.dense(inputs=dropout_hidden, units=1)
    
    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        
    optimizer = tf.train.AdamOptimizer(learning_rate, 0.9, 0.99)
    
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())

    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits, 
        train_op_fn=_train_op_fn)
  
params = {'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)}
cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        model_dir=os.path.join(model_dir, 'cnn'),
                                        params=params)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/xf/68k__xt949d59nyyhmqwx4x40000gp/T/tmps7fo4sjd/cnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a72dd7908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/xf/68k__xt949d59nyyhmqwx4x40000gp/T/tmps7fo4sjd/cnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a72dd7908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
train_and_evaluate(cnn_classifier)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


Instructions for updating:
Use keras.layers.dropout instead.


Instructions for updating:
Use keras.layers.dropout instead.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:loss = 0.36371845, step = 301 (190.133 sec)


INFO:tensorflow:loss = 0.36371845, step = 301 (190.133 sec)


INFO:tensorflow:Saving checkpoints for 320 into /var/folders/xf/68k__xt949d59nyyhmqwx4x40000gp/T/tmps7fo4sjd/cnn/model.ckpt.


INFO:tensorflow:Saving checkpoints for 320 into /var/folders/xf/68k__xt949d59nyyhmqwx4x40000gp/T/tmps7fo4sjd/cnn/model.ckpt.


KeyboardInterrupt: 